# Building All Domains with CstarSpecEngine

This notebook demonstrates how to use `CstarSpecEngine` to build all domain configurations from `domains.yml` in a single workflow.


## CstarSpecEngine Overview

`CstarSpecEngine` is the high-level interface for managing ROMS model configurations and builds. It provides methods to:

- **Generate single domains**: Build one domain at a time using `generate_domain()`
- **Generate multiple domains**: Build all domains from `domains.yml` using `generate_all()`
- **Run simulations**: Execute model runs using `run_all()`

The engine reads domain configurations from `domains.yml` and model specifications from `models.yml`, orchestrating the complete workflow from input generation through model compilation.


## Batch Processing

The `generate_all()` method processes all domains defined in `domains.yml` sequentially. This is useful for:

- **Batch workflows**: Generating multiple domain configurations in one operation
- **Consistent settings**: Applying the same parameters (clobber flags, settings overrides) to all domains
- **Error handling**: The method continues processing remaining domains even if one fails, reporting all errors at the end

Each domain goes through the complete workflow: PRECONFIG → Source Data → POSTCONFIG → BUILD → Pre-run.


## Workflow Stages

For each domain, `generate_all()` executes the complete workflow:

1. **PRECONFIG**: Initialize blueprint and grid object
2. **Source Data**: Download and prepare required datasets (GLORYS, UNIFIED, SRTM15, etc.)
3. **POSTCONFIG**: Generate all input files (grid, initial conditions, forcing)
4. **BUILD**: Render configuration templates and compile the model executable
5. **Pre-run**: Prepare for execution (partitioning, etc.)

The method returns a dictionary mapping domain names to `CstarSpecBuilder` instances, stored in `engine.builder`.


## Setup

Enable autoreload for development and import the package.


In [ ]:
%load_ext autoreload
%autoreload 2

## Generate All Domains

Create a `CstarSpecEngine` instance and generate all domains from `domains.yml`.

**Parameters:**
- `clobber_inputs`: If `True`, overwrite existing input files for all domains
- `clobber_source_data`: If `True`, re-download source datasets
- `partition_files`: If `True`, partition input files across tiles
- `test`: If `True`, truncate generation loop after 2 iterations (for testing)
- `compile_time_settings`: Optional dict of compile-time settings overrides
- `run_time_settings`: Optional dict of run-time settings overrides
- `overrides`: Optional dict of configuration overrides to apply to all domains

The method returns a dictionary mapping domain names to `CstarSpecBuilder` instances.


In [ ]:
import cson_forge

## Run All Simulations

Execute model simulations for all generated domains. The `run_all()` method handles execution for all builders and returns execution handlers for monitoring the runs.

**Note:** This cell is commented out by default. Uncomment to run all simulations.


In [ ]:
engine = cson_forge.CstarSpecEngine()
builders = engine.generate_all(clobber_inputs=True)

In [ ]:
exec_handlers = engine.run_all()    
for key, exec_handler in exec_handlers.items():
    print("-" * 100)
    print(key)
    print(exec_handler)
    print()


In [ ]:
for key, exec_handler in exec_handlers.items():
    while not handler.ExecutionStatus.is_terminal(exec_handler.status):
        print("...", end="", flush=True)
        time.sleep(30)

    if exec_handler.status == handler.ExecutionStatus.COMPLETED:
        ocn.post_run()
    else:
        raise Exception("Model run failed")